In [ ]:
import os
import json
import time
import csv
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from dotenv import load_dotenv

In [ ]:
load_dotenv(verbose=True)
TJ_SRC=os.getenv('TJ_SRC')
driver = webdriver.Chrome('./chromedriver')
driver.get(TJ_SRC)

In [ ]:
exception_list = ['악동뮤지션', '에프엑스', '오대천왕 (정형돈, 밴드 혁오)', '개리, 진실', '다이나믹 듀오, Jay Park', '일락/채린', '범키, 챈슬러, 산체스, 태완, 양다일, MXM', '크러쉬 & 로꼬', '박재범, 더블케이, 보이비', '박재범, GRAY', '아이유', '케이윌', '지드래곤', '우리 동네 음악대장']
replace_list = ['AKMU', 'F(X)', '오대천왕(정형돈,혁오)', '프라이머리, 개리, 진실', '프라이머리, 다이나믹 듀오, Jay Park', '일락(duet.채린)', '브랜뉴뮤직, 범키, 챈슬러, 산체스, 태완, 양다일, MXM', '크러쉬,로꼬', '박재범,doublek,보이비', '박재범,그레이', 'IU', 'K.Will', 'G-DRAGON', '하현우']

In [ ]:
id_list = []
music_list = []
artist_list = []
number_list = []

In [ ]:
find_dict = {
    'id': [],
    'music': [],
    'artist': [],
    'karaoke_id': [],
    'karaoke_music': [],
    'karaoke_artist': [],
}

In [ ]:
with open('itdb_tracks.csv', 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    for line in reader:
        id_list.append(line[1])
        music_list.append(line[2])
        artist_list.append(line[3])
        
with open('number.csv', 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    for line in reader:
        number_list.append(int(line[3]))
        
print(len(id_list))
print(len(number_list))

In [ ]:
def addRow(para):
    t_index = 0
    for key in find_dict.keys():
        find_dict[key].append(para[t_index])
        t_index+=1

In [ ]:
uri_base = 'http://m.tjmedia.co.kr/tjsong/song_search_result.asp?strType=1&strText='
uri_query = '&strCond=0&strSize01=100'

count = 0;
true_count = 0;

parser_list = [7, 6, 5, 4, 3, 2]

for i in range(0, len(music_list)):
    if(artist_list[i] not in exception_list):
        continue
    try:
#         print(id_list[i], music_list[i], ' // ', artist_list[i], '에 대한 검색 결과')
        find = False
        music = music_list[i]
        music = music.split('(')[0].replace(' ', '').lower()
        artist = artist_list[i]
        if artist in exception_list:
            artist = replace_list[exception_list.index(artist)]

        driver.get(uri_base + music + uri_query)

        result_list = driver.find_elements_by_css_selector('#BoardType1 tbody')[0]
        result_list = result_list.find_elements_by_css_selector('tr')
        result_list.pop(0)

        ### result_td
        ### 0: karaoke number
        ### 1: music title
        ### 2: artist name
        artist_temp = artist.replace(' ', '').lower()
#         print('비교하는 아티스트명: ', artist_temp)
        for result in result_list:
            result_td = result.find_elements_by_css_selector('td')   
            title = result_td[1]

            if len(title.find_elements_by_css_selector('*')) > 2:
                ## MR 제거
                continue
            else:
                ## 가수 맞는지 확인
                artist_search = result_td[2].text.replace(' ', '').lower()
#                 print('검색 결과: ', artist_search)
                
                if artist_search in artist_temp or artist_temp in artist_search:
                    find = True
                    true_count += 1
                    addRow([id_list[i], music_list[i], artist_list[i], result_td[0].text, result_td[1].text, result_td[2].text])
                    break
                else:
                    continue
                    
        for parser in parser_list:
            if not find:
                if len(music) > parser:
                    music = music[:parser]
                else:
                    continue
                driver.get(uri_base + music + uri_query)

                result_list = driver.find_elements_by_css_selector('#BoardType1 tbody')[0]
                result_list = result_list.find_elements_by_css_selector('tr')
                result_list.pop(0)

                for result in result_list:
                    result_td = result.find_elements_by_css_selector('td')   
                    title = result_td[1]

                    if len(title.find_elements_by_css_selector('*')) > 2:
                        ## MR 제거
                        continue
                    else:
                        ## 가수 맞는지 확인
                        artist_search = result_td[2].text.replace(' ', '').lower()

                        if artist_search in artist_temp or artist_temp in artist_search:
                            find = True
                            true_count += 1
                            addRow([id_list[i], music_list[i], artist_list[i], result_td[0].text, result_td[1].text, result_td[2].text])
                            break
                        else:
                            continue
            if find:
                break
    except:
        find = False
    if not find:
        addRow([id_list[i], music_list[i], artist_list[i], '0', '0', '0'])
    count += 1

In [ ]:
print(true_count / count)

In [ ]:
finder = 2491
print(music_list[finder])
print(artist_list[finder])

In [ ]:
len(find_dict['id'])

## 나오지 않는거
봉구&권세은
2단지
9초 -없는 한곡만
a great big world - 없는 한곡만



In [ ]:
with open('number2.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    for i in range(0, len(find_dict['id'])):
        temp = []
        for key in find_dict.keys():
            temp.append(find_dict[key][i])
        writer.writerow(temp)